<a href="https://colab.research.google.com/github/AlexandraLakka/Thesis/blob/main/Profiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
import json

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/thesis/Dataset-VM1.xlsx")
df = df.drop(df.columns[[-1]], axis=1)
df = df.drop(df.columns[[10, 11, 12]], axis=1)
df

,VM_CPU(cores),VM_RAM (bytes),Input_function,Num_of_Input,Type_of_Input,Input 1,Input 2,Size of input (bytes),Type_of_Output,Number_of_files,Max_Resident_Memory (MB),Min_Resident_Memory (MB),Avg_Resident_Memory (MB),Max_CPU_time (sec),Min_CPU_time (sec),Avg_CPU_time (sec),Total_Exec_Time (sec)
0,1,2048,merge_sort,1,list of integers,20000,None,178016,list of integers,1,19.32,19.32,19.32,0.13,0.13,0.13,0.22
1,1,2048,merge_sort,1,list of integers,50000,None,415468,list of integers,1,20.58,20.58,20.58,0.16,0.16,0.16,0.62
2,1,2048,merge_sort,1,list of integers,80000,None,651344,list of integers,1,20.50,20.50,20.50,0.14,0.14,0.14,1.23
3,1,2048,merge_sort,1,list of integers,100000,None,812456,list of integers,1,21.95,21.95,21.95,0.20,0.20,0.20,1.40
4,1,2048,merge_sort,1,list of integers,120000,None,1043552,list of integers,1,20.49,20.49,20.49,0.13,0.13,0.13,1.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1,2048,factorial,1,integer,389254,None,28,float,66,23.53,18.26,19.42,134.94,0.13,37.35,203.02
146,1,2048,factorial,1,integer,445431,None,28,float,88,24.43,18.21,19.66,187.95,0.15,51.46,276.78
147,1,2048,factorial,1,integer,496401,None,28,float,114,25.32,18.24,19.88,238.09,0.14,65.80,353.97
148,1,2048,factorial,1,integer,510200,None,28,float,115,25.43,18.35,20.00,249.30,0.14,67.02,362.51


In [ ]:
# https://stackoverflow.com/questions/14783947/grouping-clustering-numbers-in-python
def cluster(data, maxgap):
    input = []
    resource = []

    data = sorted(data, key=itemgetter(1))
    
    groups = [[data[0][1]]]
    in_groups = [[data[0][0]]]
    for x in data[1:]:
        if abs(x[1] - groups[-1][-1]) <= maxgap:
            groups[-1].append(x[1])
            in_groups[-1].append(x[0])
        else:
            groups.append([x[1]])
            in_groups.append([x[0]])
    return groups, in_groups


def get_profiling(func: list, func_input: list, file_name: str, resource: str):
  prev = 0
  difference = []

  for i in range(0, len(func_input)):
    cur = func[i]
    if i == 0:
      diff = 0
      difference.append([func_input[i],diff])
      prev = cur
    else:
      diff = cur - prev
      difference.append([func_input[i], round(diff, 2)])
      prev = cur

  data = []
  for i in range(0, len(func_input)):
    data.append([func_input[i], func[i]])

  print(data)
  print(difference)

  min = 1000
  for i in difference:
    if i[1] > 0 and i[1] == 0:
      pass
    elif i[1] > 0:
      cur = i[1]
      if cur < min:
        min = cur
    else:
      pass

  print(min)

  groups, inputs = cluster(data, min)

  resources = []
  for m in groups:
    resources.append([max(m)])

  print(groups)
  print(resources)

  profile = []

  for i in range(0, len(resources)):
    record = {}
    record[resources[i][0]] = inputs[i]
    profile.append(record)

  if resource == 'memory':
    with open(f'/content/drive/MyDrive/Colab Notebooks/thesis/data/mem_json/{file_name}.txt', 'w') as f:
      json.dump(profile, f, indent=2)
  elif resource == 'cpu':
    with open(f'/content/drive/MyDrive/Colab Notebooks/thesis/data/cpu_json/{file_name}.txt', 'w') as f:
      json.dump(profile, f, indent=2)

In [ ]:
def get_profiling_multiply(func: list, func_input_1: list, func_input_2: list, file_name: str, resource: str):
  prev = 0
  difference = []

  for i in range(0, len(func_input_1)):
    cur = func[i]
    if i == 0:
      diff = 0
      difference.append([f"{func_input_1[i]}_{func_input_2[i]}",diff])
      prev = cur
    else:
      diff = cur - prev
      difference.append([f"{func_input_1[i]}_{func_input_2[i]}", round(diff, 2)])
      prev = cur

  data = []
  for i in range(0, len(func_input_1)):
    data.append([f"{func_input_1[i]}_{func_input_2[i]}", func[i]])

  print(data)
  print(difference)

  min = 1000
  for i in difference:
    if i[1] > 0 and i[1] == 0:
      pass
    elif i[1] > 0:
      cur = i[1]
      if cur < min:
        min = cur
    else:
      pass

  print(min)

  groups, inputs = cluster(data, min)

  resources = []
  for m in groups:
    resources.append([max(m)])

  print(groups)
  print(resources)

  profile = []

  for i in range(0, len(resources)):
    record = {}
    record[resources[i][0]] = inputs[i]
    profile.append(record)

  if resource == 'memory':
    with open(f'/content/drive/MyDrive/Colab Notebooks/thesis/data/mem_json/{file_name}.txt', 'w') as f:
      json.dump(profile, f, indent=2)
  elif resource == 'cpu':
    with open(f'/content/drive/MyDrive/Colab Notebooks/thesis/data/cpu_json/{file_name}.txt', 'w') as f:
      json.dump(profile, f, indent=2)

In [ ]:
# Merge Sort
merge_sort = df[0:30]
merge_sort = merge_sort.sort_values('Size of input (bytes)')
merge_max_mem = merge_sort.iloc[:, 10]
merge_input = merge_sort.iloc[:, 5]

get_profiling(merge_max_mem, merge_input, 'merge_sort', 'memory')

# Quick Sort
quick_sort = df[30:50].reset_index(drop=True)
quick_sort = quick_sort.sort_values('Size of input (bytes)')
quick_max_mem = quick_sort.iloc[:, 10]
quick_input = quick_sort.iloc[:, 5]

get_profiling(quick_max_mem, quick_input, 'quick_sort', 'memory')

# Cholesky Matrix Decomposition
cholesky = df[60:89].reset_index(drop=True)
cholesky = cholesky.sort_values('Size of input (bytes)')
cholesky_max_mem = cholesky.iloc[:, 10]
cholesky_input = cholesky.iloc[:, 5]

get_profiling(cholesky_max_mem, cholesky_input, 'cholesky', 'memory')

#Multiply Matrix
multiply = df[90:119].reset_index(drop=True)
multiply = multiply.sort_values('Size of input (bytes)')
multiply_max_mem = multiply.iloc[:, 10]
multiply_input_1 = multiply.iloc[:, 5]
multiply_input_2 = multiply.iloc[:, 6]

get_profiling_multiply(multiply_max_mem, multiply_input_1, multiply_input_2, 'multiply', 'memory')

# Factorial
factorial = df[120:149].reset_index(drop=True)
factorial = factorial.sort_values('Size of input (bytes)')
factorial_max_mem = factorial.iloc[:, 10]
factorial_input = factorial.iloc[:, 5]

get_profiling(factorial_max_mem, factorial_input, 'factorial', 'memory')

In [ ]:
# Merge Sort
merge_sort = df[0:30]
merge_sort = merge_sort.sort_values('Size of input (bytes)')
merge_max_cpu = merge_sort.iloc[:, 13]
merge_input = merge_sort.iloc[:, 5]

get_profiling(merge_max_cpu, merge_input, 'merge_sort', 'cpu')

# Quick Sort
quick_sort = df[30:50].reset_index(drop=True)
quick_sort = quick_sort.sort_values('Size of input (bytes)')
quick_max_cpu = quick_sort.iloc[:, 13]
quick_input = quick_sort.iloc[:, 5]

get_profiling(quick_max_cpu, quick_input, 'quick_sort', 'cpu')

# Cholesky Matrix Decomposition
cholesky = df[60:89].reset_index(drop=True)
cholesky = cholesky.sort_values('Size of input (bytes)')
cholesky_max_cpu = cholesky.iloc[:, 13]
cholesky_input = cholesky.iloc[:, 5]

get_profiling(cholesky_max_cpu, cholesky_input, 'cholesky', 'cpu')

#Multiply Matrix
multiply = df[90:119].reset_index(drop=True)
multiply = multiply.sort_values('Size of input (bytes)')
multiply_max_cpu = multiply.iloc[:, 13]
multiply_input_1 = multiply.iloc[:, 5]
multiply_input_2 = multiply.iloc[:, 6]

get_profiling_multiply(multiply_max_cpu, multiply_input_1, multiply_input_2, 'multiply', 'cpu')

# Factorial
factorial = df[120:149].reset_index(drop=True)
factorial = factorial.sort_values('Size of input (bytes)')
factorial_max_cpu = factorial.iloc[:, 13]
factorial_input = factorial.iloc[:, 5]

get_profiling(factorial_max_cpu, factorial_input, 'factorial', 'cpu')
